In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
from tenacity import retry, stop_after_attempt, wait_exponential
import time

In [ ]:
urls_df = pd.read_csv('cleaned_sale_links.csv')
urls_df['Links'] = urls_df['Links'].str.strip()  

In [ ]:
empty_urls = urls_df[urls_df['Links'].isna() | (urls_df['Links'] == '')]
if not empty_urls.empty:
    print("Empty URLs found:")
    print(empty_urls)

urls = urls_df['Links'].dropna().tolist()

In [ ]:
data = []

@retry(stop=stop_after_attempt(5), wait=wait_exponential(multiplier=1, min=4, max=10), reraise=True)
def fetch_page(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    response = requests.get(url, headers=headers, timeout=10)  
    response.raise_for_status()  
    return response

def scrape_url(url):
    if url == '':
        print("Empty URL found")
        return [None, None, None, None, url]

    try:
        page = fetch_page(url) 

        soup = BeautifulSoup(page.text, 'html.parser')

        span_elements = soup.find_all('span')
        h5_elements = soup.find_all('h5')
        paragraphs = soup.find('div', id='collapsefour')

        Number = None
        Date = None
        City = None
        Fact = None


        for span in span_elements:
            text = span.get_text().strip()
            if Number is None and (text.startswith('رقم القضية - القرار:') or text.startswith('رقم القرار:')):
                Number = text.replace('رقم القضية - القرار:', '').replace('رقم القرار:', '').strip()
            elif Date is None and (text.startswith('تاريخها:') or text.startswith('تاريخه:')):
                Date = text.replace('تاريخها:', '').replace('تاريخه:', '').strip()
            if Number and Date:
                break


        for h5 in h5_elements:
            text = h5.get_text().strip()
            if City is None and (text.startswith('المدينه:') or text.startswith('المدينة:')):
                City = text.replace('المدينه:', '').replace('المدينة:', '').strip()
                break

        # Extract Fact
        if paragraphs:
            Fact = paragraphs.get_text().strip()
        else:
            fact_element_alt = soup.find('div', id='collapsefive')
            if fact_element_alt:
                Fact = fact_element_alt.get_text().strip()

        return [Number, Date, City, Fact, url]

    except requests.exceptions.RequestException as e:
        print(f"Network error processing {url}: {e}")
        return [None, None, None, None, url]
    except Exception as e:
        print(f"Error processing {url}: {e}")
        return [None, None, None, None, url]

with ThreadPoolExecutor(max_workers=10) as executor:
    results = list(executor.map(scrape_url, urls))

data.extend(results)

df = pd.DataFrame(data, columns=['Number', 'Date', 'City', 'Fact', 'URL'])
df.to_csv('Sale_Case_Data.csv', index=False)

print("Data saved successfully to 'Sale_Case_Data.csv'.")

In [6]:
df = pd.read_csv('Sale_Case_Data.csv', engine='python')
df

,Number,Date,City,Fact,URL
0,٤٥٣٠١٨٧٤٩٩,٥/٣/٢٤٤٥,جدة,الحمدلله والصلاة والسلام على رسول الله، أما بع...,https://sjp.moj.gov.sa/Filter/AhkamDetails/68871
1,٤٦٣٠١١٩٠٦٨,٩/١٠/١٤٤٥,جدة,الحمد لله والصلاة والسلام على رسول الله، أما ب...,https://sjp.moj.gov.sa/Filter/AhkamDetails/79311
2,٤٦٣٠٢١١٧٥٥,١٦/٣/١٤٤٦,الرياض,الحمد لله والصلاة والسلام على رسول الله، أما ب...,https://sjp.moj.gov.sa/Filter/AhkamDetails/82190
3,٤٦٣٠٢١٧٠٣١,١٤/٣/١٤٤٦,الدمام,الحمدلله والصلاة والسلام على رسول الله، أما بع...,https://sjp.moj.gov.sa/Filter/AhkamDetails/82450
4,٤٦٣٠٢٣١٩٥٧,١٤/٣/١٤٤٦,الرياض,الحمدلله والصلاة والسلام على رسول الله، أما بع...,https://sjp.moj.gov.sa/Filter/AhkamDetails/82532
...,...,...,...,...,...
8296,٧٧٥١/٣/ق,٢٨ / ٦ / ١٤٣٨,الدمام,الحكم في القضية رقم ٧٧٥١/٣/ق لعام ١٤٣٨هـالمقام...,https://sjp.moj.gov.sa/Filter/AhkamDetails/13325
8297,٧٩٢٩/٢/ق,٢٩ / ٦ / ١٤٣٨,جدة,الحكم في القضية رقم ٧٩٢٩/٢/ق لعام ١٤٣٤هــالمقا...,https://sjp.moj.gov.sa/Filter/AhkamDetails/13348
8298,١٠٧٩٤/١/ق,٢٢ / ٨ / ١٤٣٨,الرياض,حكم في القضية رقم ١٠٧٩٤/١/ق لعام ١٤٣٦هـالمقامة...,https://sjp.moj.gov.sa/Filter/AhkamDetails/13194
8299,٤٣٣٩/٣/ق,١ / ١١ / ١٤٣٨,الدمام,حكم في القضية رقم ٤٣٣٩/٣/ق لعام ١٤٣٨هـالمقامة ...,https://sjp.moj.gov.sa/Filter/AhkamDetails/13400


In [7]:
df.isnull().sum()

,0
Number,6
Date,166
City,361
Fact,361
URL,556
